In [1]:
from utilities.utilities import load_data, get_records_by_region, create_column, finalize_dataframe, get_extreme_values, create_directory_structure, save_table, save_report, pd, assign_quartile
# settings
region_column_name = 'Region'
table_name = 'https_by_region'
report_name = 'https_by_region'
category = 'https'
column_name_to_results_global = 'Global #'
create_directory_structure()

source_df = load_data('https_checker')

In [6]:
source_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458 entries, 0 to 457
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   id                             458 non-null    int64 
 1   region                         458 non-null    object
 2   name                           458 non-null    object
 3   category                       458 non-null    object
 4   url                            458 non-null    object
 5   has_http                       458 non-null    bool  
 6   has_https                      458 non-null    bool  
 7   forced_redirect_to_https       458 non-null    bool  
 8   https_redirect_to_same_domain  458 non-null    bool  
 9   only_https                     458 non-null    bool  
dtypes: bool(5), int64(1), object(4)
memory usage: 20.3+ KB


In [2]:
# sanity dataset
columns_names = ['has_http', 'has_https', 'forced_redirect_to_https', 'https_redirect_to_same_domain', 'only_https']
for column in columns_names:
    source_df.loc[source_df[column].isna(), column] = False

In [3]:
# Analyze of HEIs with HTTPS by region (Pub/Pvt)


# settings
column_to_sort = 'HTTP Only %'
sort_ascending = True
config = [
    {'table_name': 'https_by_region_public', 'hei_type': 'Public'},
    {'table_name': 'https_by_region_private', 'hei_type': 'Private'}
]
dfs = []
for config_item in config:
    table_name = config_item['table_name']
    hei_type = config_item['hei_type']
    columns_to_display = [region_column_name.title(), column_name_to_results_global]
    analysis_df = get_records_by_region(source_df, hei_type=hei_type)
    # create columns
    # Column creation with distribution of records invalid by region
    criteria = 'has_http == False & has_https == False & forced_redirect_to_https == False & https_redirect_to_same_domain == False &  only_https == False'
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Invalid', criteria=f'{criteria} & category == "{hei_type}"', columns_to_display=columns_to_display)
    # Column creation with distribution of records with only HTTP by region
    criteria = 'has_http == True & has_https == False & forced_redirect_to_https == False & only_https == False'
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='HTTP Only', criteria=f'{criteria} & category == "{hei_type}"', columns_to_display=columns_to_display)
    # Column creation with distribution of records with HTTPS without redirect by region
    criteria = 'has_http == True & has_https == True & forced_redirect_to_https == False & only_https == False & https_redirect_to_same_domain == False'
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='HTTP and HTTPS', criteria=f'{criteria} & category == "{hei_type}"', columns_to_display=columns_to_display)
    # Column creation with distribution of records with forced redirect to HTTPS (same domain) by region
    criteria = 'has_http == True & has_https == True & forced_redirect_to_https == True & https_redirect_to_same_domain == True'
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='HTTP to HTTPS (Same Domain)', criteria=f'{criteria} & category == "{hei_type}"', columns_to_display=columns_to_display)
    # Column creation with distribution of records with forced redirect to HTTPS (other domain) by region
    criteria = 'has_http == True & has_https == True & forced_redirect_to_https == True & https_redirect_to_same_domain == False'
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='HTTP to HTTPS (Other Domain)', criteria=f'{criteria} & category == "{hei_type}"', columns_to_display=columns_to_display)
    # Column creation with distribution of records with only HTTPS by region
    criteria = 'has_http == False & has_https == True & only_https == True'
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='HTTPS Only', criteria=f'{criteria} & category == "{hei_type}"', columns_to_display=columns_to_display)
    # Finalize dataframe
    analysis_df = finalize_dataframe(dataframe=analysis_df, column_to_sort=column_to_sort, ascending=sort_ascending, columns_to_display=columns_to_display)
    display(analysis_df)
    dfs.append(analysis_df)
    # save to csv
    save_table(analysis_df, category=category, table_name=table_name)


df_public = dfs[0].add_suffix('(pub)')
df_private = dfs[1].add_suffix('(pvt)')
df_public = df_public.rename(columns={'Region(pub)': 'Region'})
df_private = df_private.rename(columns={'Region(pvt)': 'Region'})
df_combined = df_public.merge(df_private, on='Region', how='outer')
df_combined.fillna(0, inplace=True)
# remove columns with # in the name
df_combined = df_combined.loc[:, ~df_combined.columns.str.contains('#')]
#remove columns global
df_combined = df_combined.loc[:, ~df_combined.columns.str.contains('Global')]
# remove '%' from name of columns
df_combined.columns = df_combined.columns.str.replace('%', '')

ranks_columns = ['Rank', 'Rank2', 'Rank3', 'Rank4']
#add column with sum of best columns
df_combined[ranks_columns[0]] = df_combined['HTTPS Only (pub)'] + df_combined['HTTPS Only (pvt)']
df_combined[ranks_columns[1]] = df_combined['HTTP to HTTPS (Same Domain) (pub)'] + df_combined['HTTP to HTTPS (Same Domain) (pvt)']
df_combined[ranks_columns[2]] = df_combined['HTTP to HTTPS (Other Domain) (pub)'] + df_combined['HTTP to HTTPS (Other Domain) (pvt)']
df_combined[ranks_columns[3]] = df_combined['HTTP and HTTPS (pub)'] + df_combined['HTTP and HTTPS (pvt)']


#order dataframe by column Rank (from highest to lowest)
df_combined = df_combined.sort_values(by=ranks_columns, ascending=False)
# move just row with 'Total' in column Region to the end of the dataframe. (Use pandas.concat instead of append to avoid duplicates)
df_combined = pd.concat([df_combined[df_combined['Region'] != 'Total'], df_combined[df_combined['Region'] == 'Total']])
# reset index
df_combined.reset_index(drop=True, inplace=True)
# remove column Rank
df_combined.drop(columns=ranks_columns, inplace=True)
#Add a column with the quartile corresponding to the position of the row, that is, considering the total of records -1 (to exclude the total row), if a row is in position 2 it should belong to the first quartile.
df_combined['Quartile'] = df_combined.index.map(lambda rank: assign_quartile(rank, len(df_combined)-1))
# moved column 'Quartile' to the second position
cols = list(df_combined.columns)
cols = [cols[0]] + [cols[-1]] + cols[1:-1]
df_combined = df_combined[cols]


save_table(df_combined, category=category, table_name='https_by_region_combined')

,Region,Global #,Invalid #,Invalid %,HTTP Only #,HTTP Only %,HTTP and HTTPS #,HTTP and HTTPS %,HTTP to HTTPS (Same Domain) #,HTTP to HTTPS (Same Domain) %,HTTP to HTTPS (Other Domain) #,HTTP to HTTPS (Other Domain) %,HTTPS Only #,HTTPS Only %
0,Baden-Württemberg,48,0,0.0,0,0.000000,2,4.166667,45,93.750000,0,0.000000,1,2.083333
1,Bayern,34,0,0.0,0,0.000000,0,0.000000,34,100.000000,0,0.000000,0,0.000000
2,Berlin,12,0,0.0,0,0.000000,0,0.000000,12,100.000000,0,0.000000,0,0.000000
3,Brandenburg,10,0,0.0,0,0.000000,0,0.000000,10,100.000000,0,0.000000,0,0.000000
4,Bremen,5,0,0.0,0,0.000000,0,0.000000,4,80.000000,1,20.000000,0,0.000000
5,Hamburg,10,0,0.0,0,0.000000,0,0.000000,9,90.000000,0,0.000000,1,10.000000
6,Hessen,22,0,0.0,0,0.000000,1,4.545455,21,95.454545,0,0.000000,0,0.000000
7,Nordrhein-Westfalen,42,0,0.0,0,0.000000,0,0.000000,40,95.238095,2,4.761905,0,0.000000
8,Rheinland-Pfalz,15,0,0.0,0,0.000000,0,0.000000,15,100.000000,0,0.000000,0,0.000000
9,Saarland,5,0,0.0,0,0.000000,0,0.000000,5,100.000000,0,0.000000,0,0.000000


,Region,Global #,Invalid #,Invalid %,HTTP Only #,HTTP Only %,HTTP and HTTPS #,HTTP and HTTPS %,HTTP to HTTPS (Same Domain) #,HTTP to HTTPS (Same Domain) %,HTTP to HTTPS (Other Domain) #,HTTP to HTTPS (Other Domain) %,HTTPS Only #,HTTPS Only %
0,Baden-Württemberg,24,0,0.0,0,0.0,1,4.166667,23,95.833333,0,0.000000,0,0.0
1,Bayern,12,0,0.0,0,0.0,0,0.000000,12,100.000000,0,0.000000,0,0.0
2,Berlin,28,0,0.0,0,0.0,2,7.142857,26,92.857143,0,0.000000,0,0.0
3,Brandenburg,9,0,0.0,0,0.0,1,11.111111,8,88.888889,0,0.000000,0,0.0
4,Bremen,2,0,0.0,0,0.0,0,0.000000,2,100.000000,0,0.000000,0,0.0
5,Hamburg,11,0,0.0,0,0.0,1,9.090909,9,81.818182,1,9.090909,0,0.0
6,Hessen,23,0,0.0,0,0.0,0,0.000000,21,91.304348,2,8.695652,0,0.0
7,Niedersachsen,16,0,0.0,0,0.0,1,6.250000,14,87.500000,1,6.250000,0,0.0
8,Nordrhein-Westfalen,25,0,0.0,0,0.0,1,4.000000,24,96.000000,0,0.000000,0,0.0
9,Rheinland-Pfalz,6,0,0.0,0,0.0,1,16.666667,3,50.000000,2,33.333333,0,0.0


In [4]:
# Analyze of HEIs with HTTPS by region

# settings
column_to_sort = 'HTTP Only (Public) %'
sort_ascending = True
columns_to_display = [region_column_name, column_name_to_results_global]
analysis_df = get_records_by_region(source_df)

# create columns
only_public = 'category == "Public"'
only_private = 'category == "Private"'

# Column creation with distribution of records invalid by region
criteria = 'has_http == False & has_https == False & forced_redirect_to_https == False & https_redirect_to_same_domain == False &  only_https == False'
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Invalid', criteria=f'{criteria}', columns_to_display=columns_to_display)

# Column creation with distribution of records with only HTTP by region
criteria = 'has_http == True & has_https == False & forced_redirect_to_https == False & only_https == False'
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='HTTP Only (Public)', criteria=f'{criteria} & {only_public}', columns_to_display=columns_to_display)
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='HTTP Only (Private)', criteria=f'{criteria} & {only_private}', columns_to_display=columns_to_display)

# Column creation with distribution of records with HTTPS without redirect by region
criteria = 'has_http == True & has_https == True & forced_redirect_to_https == False & only_https == False & https_redirect_to_same_domain == False'
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='HTTP and HTTPS (Public)', criteria=f'{criteria} & {only_public}', columns_to_display=columns_to_display)
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='HTTP and HTTPS (Private)', criteria=f'{criteria} & {only_private}', columns_to_display=columns_to_display)

# Column creation with distribution of records with forced redirect to HTTPS (same domain) by region
criteria = 'has_http == True & has_https == True & forced_redirect_to_https == True & https_redirect_to_same_domain == True'
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='HTTP to HTTPS (Same Domain) (Public)', criteria=f'{criteria} & {only_public}', columns_to_display=columns_to_display)
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='HTTP to HTTPS (Same Domain) (Private)', criteria=f'{criteria} & {only_private}', columns_to_display=columns_to_display)

# Column creation with distribution of records with forced redirect to HTTPS (other domain) by region
criteria = 'has_http == True & has_https == True & forced_redirect_to_https == True & https_redirect_to_same_domain == False'
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='HTTP to HTTPS (Other Domain) (Public)', criteria=f'{criteria} & {only_public}', columns_to_display=columns_to_display)
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='HTTP to HTTPS (Other Domain) (Private)', criteria=f'{criteria} & {only_private}', columns_to_display=columns_to_display)

# Column creation with distribution of records with only HTTPS by region
criteria = 'has_http == False & has_https == True & only_https == True'
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='HTTPS Only (Public)', criteria=f'{criteria} & {only_public}', columns_to_display=columns_to_display)
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='HTTPS Only (Private)', criteria=f'{criteria} & {only_private}', columns_to_display=columns_to_display)



# Finalize dataframe
analysis_df = finalize_dataframe(dataframe=analysis_df, column_to_sort=column_to_sort, ascending=sort_ascending, columns_to_display=columns_to_display)
display(analysis_df)

# save to csv
save_table(analysis_df, category=category, table_name=table_name)


,Region,Global #,Invalid #,Invalid %,HTTP Only (Public) #,HTTP Only (Public) %,HTTP Only (Private) #,HTTP Only (Private) %,HTTP and HTTPS (Public) #,HTTP and HTTPS (Public) %,...,HTTP to HTTPS (Same Domain) (Private) #,HTTP to HTTPS (Same Domain) (Private) %,HTTP to HTTPS (Other Domain) (Public) #,HTTP to HTTPS (Other Domain) (Public) %,HTTP to HTTPS (Other Domain) (Private) #,HTTP to HTTPS (Other Domain) (Private) %,HTTPS Only (Public) #,HTTPS Only (Public) %,HTTPS Only (Private) #,HTTPS Only (Private) %
0,Alabama,32,0,0.000000,0,0.000000,1,3.125000,0,0.000000,...,16,50.000000,0,0.000000,0,0.000000,0,0.000000,0,0.000000
1,Nebraska,21,0,0.000000,0,0.000000,0,0.000000,0,0.000000,...,13,61.904762,0,0.000000,1,4.761905,0,0.000000,0,0.000000
2,Nevada,10,0,0.000000,0,0.000000,0,0.000000,0,0.000000,...,7,70.000000,0,0.000000,0,0.000000,0,0.000000,0,0.000000
3,New Hampshire,14,0,0.000000,0,0.000000,0,0.000000,0,0.000000,...,7,50.000000,0,0.000000,0,0.000000,0,0.000000,0,0.000000
4,New Jersey,48,0,0.000000,0,0.000000,0,0.000000,0,0.000000,...,33,68.750000,0,0.000000,0,0.000000,0,0.000000,0,0.000000
5,New Mexico,10,0,0.000000,0,0.000000,0,0.000000,2,20.000000,...,3,30.000000,0,0.000000,0,0.000000,0,0.000000,0,0.000000
6,New York,175,1,0.571429,0,0.000000,1,0.571429,3,1.714286,...,113,64.571429,0,0.000000,1,0.571429,0,0.000000,1,0.571429
7,North Carolina,63,0,0.000000,0,0.000000,0,0.000000,1,1.587302,...,44,69.841270,0,0.000000,0,0.000000,1,1.587302,0,0.000000
8,North Dakota,9,0,0.000000,0,0.000000,0,0.000000,0,0.000000,...,2,22.222222,0,0.000000,0,0.000000,0,0.000000,0,0.000000
9,Ohio,74,0,0.000000,0,0.000000,0,0.000000,3,4.054054,...,54,72.972973,0,0.000000,0,0.000000,0,0.000000,0,0.000000


In [5]:
# Report in latex
report_results = get_extreme_values(analysis_df)

hei_public_invalid = format(report_results.get("Total").get("Invalid %"), ".2f")

hei_public_only_http = format(report_results.get("Total").get("HTTP Only (Public) %"), ".2f")
hei_public_only_https = format(report_results.get("Total").get("HTTPS Only (Public) %"), ".2f")
hei_public_http_and_https = format(report_results.get("Total").get("HTTP and HTTPS (Public) %"), ".2f")
hei_public_redirect_same = format(report_results.get("Total").get("HTTP to HTTPS (Same Domain) (Public) %"), ".2f")
hei_public_redirect_other = format(report_results.get("Total").get("HTTP to HTTPS (Other Domain) (Public) %"), ".2f")

hei_private_only_http = format(report_results.get("Total").get("HTTP Only (Private) %"), ".2f")
hei_private_only_https = format(report_results.get("Total").get("HTTPS Only (Private) %"), ".2f")
hei_private_http_and_https = format(report_results.get("Total").get("HTTP and HTTPS (Private) %"), ".2f")
hei_private_redirect_same = format(report_results.get("Total").get("HTTP to HTTPS (Same Domain) (Private) %"), ".2f")
hei_private_redirect_other = format(report_results.get("Total").get("HTTP to HTTPS (Other Domain) (Private) %"), ".2f")

report_figure = """
\\begin{figure}[htbp]
    \centering
    \includegraphics[width=0.48\\textwidth]{charts/https_by_region.pdf}
    \caption{Distribution of HTTPS by regions.}\label{fig:https}
\end{figure}
"""

report = f'{report_figure}\n\n'
report += f"""
Fig.~\\ref{{fig:https}} presents an overview of the use of \gls{{http}}/\gls{{https}} services at \glspl{{hei}} in \countryName. The indicators represent the following conditions:

\\begin{{itemize}}
    \item Invalid: The institution website doesn't work.
    \item \gls{{http}} only: The institution website uses only \gls{{http}}.
    \item \gls{{http}} {{\&}} \gls{{https}}: The institution offers both protocols but does not have any redirection to force the use of \gls{{https}}. All institutions in this category have a valid certificate.
    \item  \gls{{http}} to \gls{{https}} (other): Institution website that redirects the user to a secure page outside the main domain. All institutions in this category have a valid certificate.
    \item \gls{{http}} to \gls{{https}} (same): Institutions website that redirects the user to a secure page within the main domain, thus ensuring data protection. All institutions in this category have a valid certificate.
    \item \gls{{http}} only: The institution website uses only \gls{{http}}.
\end{{itemize}}\n\n"""

report += f"""
According to the data, {hei_public_only_http}\% of the public institutions analyzed have not implemented \gls{{https}} on their websites, while {hei_private_only_http}\% of the private institutions analyzed also dont use \gls{{https}}.

On a positive note, {hei_public_redirect_same}\% of the public institutions analyzed have implemented forced redirect to same domain, and {hei_public_redirect_other}\% to other domain. While {hei_private_redirect_same}\% of the private institutions have implemented forced redirect to same domain, and {hei_private_redirect_other}\% to other domain. And {hei_public_only_https}\% of the public institutions, and {hei_private_only_https}\% of the private institutions have implemented only \gls{{https}} on their websites, ensuring data protection.

However, {hei_public_http_and_https}\% of the public institutions, and {hei_private_http_and_https}\% of the private institutions keeps both protocols active, allowing the user to access the website through the \gls{{http}} protocol.\n\n"""

report += f"""
In terms of regional differences, private institutions in {report_results.get("HTTP to HTTPS (Same Domain) (Private) %").get("top_regions")[0][0]} ({format(report_results.get("HTTP to HTTPS (Same Domain) (Private) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("HTTP to HTTPS (Same Domain) (Private) %").get("top_regions")[1][0]} ({format(report_results.get("HTTP to HTTPS (Same Domain) (Private) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("HTTP to HTTPS (Same Domain) (Private) %").get("top_regions")[2][0]} ({format(report_results.get("HTTP to HTTPS (Same Domain) (Private) %").get("top_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("HTTP to HTTPS (Same Domain) (Public) %").get("top_regions")[0][0]} ({format(report_results.get("HTTP to HTTPS (Same Domain) (Public) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("HTTP to HTTPS (Same Domain) (Public) %").get("top_regions")[1][0]} ({format(report_results.get("HTTP to HTTPS (Same Domain) (Public) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("HTTP to HTTPS (Same Domain) (Public) %").get("top_regions")[2][0]} ({format(report_results.get("HTTP to HTTPS (Same Domain) (Public) %").get("top_regions")[2][1], ".2f")}\%) have a higher usage of \gls{{https}} with forced redirect to same domain.

In contrast, private institutions in {report_results.get("HTTP to HTTPS (Same Domain) (Private) %").get("bottom_regions")[0][0]} ({format(report_results.get("HTTP to HTTPS (Same Domain) (Private) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("HTTP to HTTPS (Same Domain) (Private) %").get("bottom_regions")[1][0]} ({format(report_results.get("HTTP to HTTPS (Same Domain) (Private) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("HTTP to HTTPS (Same Domain) (Private) %").get("bottom_regions")[2][0]} ({format(report_results.get("HTTP to HTTPS (Same Domain) (Private) %").get("bottom_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("HTTP to HTTPS (Same Domain) (Public) %").get("bottom_regions")[0][0]} ({format(report_results.get("HTTP to HTTPS (Same Domain) (Public) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("HTTP to HTTPS (Same Domain) (Public) %").get("bottom_regions")[1][0]} ({format(report_results.get("HTTP to HTTPS (Same Domain) (Public) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("HTTP to HTTPS (Same Domain) (Public) %").get("bottom_regions")[2][0]} ({format(report_results.get("HTTP to HTTPS (Same Domain) (Public) %").get("bottom_regions")[2][1], ".2f")}\%) have a lower usage of \gls{{https}} with forced redirect to same domain.

Finally, private institutions in {report_results.get("HTTP and HTTPS (Private) %").get("top_regions")[0][0]} ({format(report_results.get("HTTP and HTTPS (Private) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("HTTP and HTTPS (Private) %").get("top_regions")[1][0]} ({format(report_results.get("HTTP and HTTPS (Private) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("HTTP and HTTPS (Private) %").get("top_regions")[2][0]} ({format(report_results.get("HTTP and HTTPS (Private) %").get("top_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("HTTP and HTTPS (Public) %").get("top_regions")[0][0]} ({format(report_results.get("HTTP and HTTPS (Public) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("HTTP and HTTPS (Public) %").get("top_regions")[1][0]} ({format(report_results.get("HTTP and HTTPS (Public) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("HTTP and HTTPS (Public) %").get("top_regions")[2][0]} ({format(report_results.get("HTTP and HTTPS (Public) %").get("top_regions")[2][1], ".2f")}\%) have a higher usage of both protocols \gls{{dnssec}}, allowing the user to access the website through the \gls{{http}} protocol."""

print(report)
# save report to file txt
save_report(report=report, category=category, report_name=report_name)


\begin{figure}[htbp]
    \centering
    \includegraphics[width=0.48\textwidth]{charts/https_by_region.pdf}
    \caption{Distribution of HTTPS by regions.}\label{fig:https}
\end{figure}



Fig.~\ref{fig:https} presents an overview of the use of \gls{http}/\gls{https} services at \glspl{hei} in \countryName. The indicators represent the following conditions:

\begin{itemize}
    \item Invalid: The institution website doesn't work.
    \item \gls{http} only: The institution website uses only \gls{http}.
    \item \gls{http} {\&} \gls{https}: The institution offers both protocols but does not have any redirection to force the use of \gls{https}. All institutions in this category have a valid certificate.
    \item  \gls{http} to \gls{https} (other): Institution website that redirects the user to a secure page outside the main domain. All institutions in this category have a valid certificate.
    \item \gls{http} to \gls{https} (same): Institutions website that redirects the user to a sec